# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
import os
import seaborn as sns
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

#### Path management

In [ ]:
base: str
if os.getcwd() == "/kaggle/working":
    base = "/kaggle"
else:
    base = os.path.join(os.getcwd())

def get_full_dir(sub_dir: str) -> str:
    return os.path.join(base, sub_dir)

# EDA

In [ ]:
df_sample_submission = pd.read_csv(get_full_dir('input/playground-series-s3e14/sample_submission.csv'))
df_train = pd.read_csv(get_full_dir('input/playground-series-s3e14/train.csv'), index_col='id')
df_test = pd.read_csv(get_full_dir('input/playground-series-s3e14/test.csv'), index_col='id')

In [ ]:
df_train.isna().sum()

##### There are no missing values in our train data.

In [ ]:
df_sample_submission.head()

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
fig, axes = plt.subplots(nrows=len(df_train.select_dtypes(include='number').columns), ncols=4, figsize=(18, 80))
axes = axes.flatten()

i = 0
for col in df_train.select_dtypes(include='number').columns:
    sns.kdeplot(df_train[col], label='train', ax=axes[i], fill=False)
    sns.histplot(df_train[col], label='train', ax=axes[i + 1], stat="density", bins=50)

    if col != 'yield':
        sns.kdeplot(df_test[col], label='test', ax=axes[i], fill=False)
        sns.histplot(df_test[col], label='test', ax=axes[i + 1], stat="density", bins=50)

    if col != 'yield':
        tmp_data = pd.DataFrame({"train": df_train[col], "test": df_test[col]})
        sns.boxplot(data=tmp_data, ax=axes[i + 2])
    else:
        tmp_data = pd.DataFrame({"train": df_train[col]})
        sns.boxplot(data=tmp_data, ax=axes[i + 2])
    axes[i + 2].set_xlabel(col)

    sns.scatterplot(x=col, y="yield", label='train', ax=axes[i + 3], data=df_train)

    axes[i].legend()
    axes[i + 1].legend()
    axes[i + 3].legend()
    i += 4

plt.show()

##### The apper to be some true outliers in the honeybee colum around 18 and 6, the mean for that column is ~0.389

In [ ]:
def show_honeybee_outlier_count(df: pd.DataFrame):
    honeybee_data = df['honeybee']
    total = honeybee_data.count()
    below_one = honeybee_data[honeybee_data < 1].count()
    above_17 = honeybee_data[honeybee_data > 17].count()
    one_to_17 = honeybee_data[honeybee_data > 1].count() - above_17
    print(f" Total:\t\t {total}")
    print(f" x < 1:\t\t {below_one}")
    print(f" 1 < x < 17: {one_to_17}")
    print(f" 17 < x:\t {above_17}")

print("Train")
show_honeybee_outlier_count(df_train)
print("Test")
show_honeybee_outlier_count(df_test)

##### Although the Train and Test datasets have a relatively small number of outliers (8 and 6, respectively), representing less than 0.1% of the datasets, it may still be beneficial to remove them. While outliers can sometimes be genuine observations, they can also have a significant impact on statistical analysis and machine learning models, leading to biased results and poor predictive performance. (Removing the outliers from the train set has improved both evaluation loss and loss on the public set upon kaggle submission)

In [ ]:
def show_feature_correlation(df: pd.DataFrame, title: str):
    plt.figure(figsize=(20, 20))
    corr_matrix = df.corr()

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr_matrix, dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, mask=mask)
    plt.title(title)
    plt.show()

show_feature_correlation(df_train, "Train")
show_feature_correlation(df_test, "Test")

##### MaxOfUpperTRange, MinOfUpperTRange, AverageOfUpperTRange, MaxOfLowerTRange, MinOfLowerTRange, AverageOfLowerTRange are all perfectly correlated and AverageRainingDays is almost perfectly correlated to RainingDays.

##### When two or more features in a dataset are highly correlated, they can provide redundant information to the model, which does not add any additional information over the other features. This redundancy can cause instability in the model and lead to biased predictions.

##### Decision tree-based algorithms, such as Random Forest, LGBM and XGBoost, choose a subset of features for consideration at each node. This situation can lead to a bias towards the correlated features, which may negatively impact the model's performance.

##### To mitigate this issue we will remove all but one of the perfectly correlated features. We will remove RainingDays leaving AverageRainingDays and MaxOfUpperTRange, MinOfUpperTRange, AverageOfUpperTRange, MaxOfLowerTRange and MinOfLowerTRange leaving AverageOfLowerTRange


# Prepare data for training

In [ ]:
# Handle outliers
df_train = df_train[df_train['honeybee'] < 1]
df_train.reset_index(drop=True, inplace=True)
#df_test[df_test['honeybee'] > 1] = df_train['honeybee'].mean()

# Remove perfectly correlated features
features_to_remove = ['RainingDays', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange']
df_train.drop(features_to_remove, axis=1)
df_test.drop(features_to_remove, axis=1)

# Scale features
standard_scaler = StandardScaler()
X = df_train.drop(columns=['yield'])
X_scaled = standard_scaler.fit_transform(X)
df_train = pd.concat([pd.DataFrame(X_scaled, columns=X.columns), df_train['yield']], axis=1)
X_scaled = standard_scaler.transform(df_test)
df_test = pd.DataFrame(X_scaled, columns=X.columns)

# Train Models

In [ ]:
param_dist_lgbm = {
    'n_estimators': [1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 5, 10, 15, 20],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'reg_alpha': [0, 0.1, 0.2, 0.3, 1],
    'reg_lambda': [0, 0.1, 0.2, 0.3, 1],
    'num_leaves': [12, 16, 20],
    'objective': ["mae"]
}

param_dist_catboost = {
    'iterations': [1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 5, 6, 7],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bylevel': [0.5, 0.7, 0.9],
    'random_strength': [0.1, 0.5, 1],
    'bagging_temperature': [0.1, 0.5, 1],
    'loss_function': ['MAE'],
    'verbose': [False]
}

param_dist_xgboost = {
    'n_estimators': [1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 5, 10, 15, 20],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'gamma': [0, 0.1, 0.2, 0.3, 1],
    'reg_alpha': [0, 0.1, 0.2, 0.3, 1],
    'reg_lambda': [0, 0.1, 0.2, 0.3, 1],
    'objective': ["reg:squarederror", 'reg:absoluteerror'],
    'eval_metric': ['mae'],
}

In [ ]:
def param_search(model, param_space, n_iter) -> RandomizedSearchCV:
    # Load data
    X_train = pd.read_csv(get_full_dir('input/playground-series-s3e14/train.csv'), index_col='id')
    y_train = X_train.pop('yield')

    # Create RandomizedSearchCV object
    rs = RandomizedSearchCV(model, param_distributions=param_space, cv=5, n_iter=n_iter, scoring='neg_mean_absolute_error')
    # Fit the model on the training data
    rs.fit(X_train, y_train)
    # Return results
    return rs

In [ ]:
# Param searches the space
search = False

In [ ]:
if search:
    # Defines models
    lgb_model = lgb.LGBMRegressor()
    cat_model = CatBoostRegressor()
    xgb_model = xgb.XGBRegressor(tree_method = 'gpu_hist')

    lgb_rs = param_search(lgb_model, param_dist_lgbm, 100)
    print("Best LGBM parameters:", lgb_rs.best_params_)
    print("Best LGBM MAE score:", -lgb_rs.best_score_)
    lgb_param = lgb_rs.best_params_

    cat_rs = param_search(cat_model, param_dist_catboost, 100)
    print("Best CatBoost parameters:", cat_rs.best_params_)
    print("Best CatBoost MAE score:", -cat_rs.best_score_)
    cat_param = cat_rs.best_params_

    xgb_rs = param_search(xgb_model, param_dist_xgboost, 100)
    print("Best XGBoost parameters:", xgb_rs.best_params_)
    print("Best XGBoost MAE score:", -xgb_rs.best_score_)
    xgb_param = xgb_rs.best_params_
else:
    # Best LGBM MAE score: 343.512189589491
    lgb_param = {'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0.3, 'objective': 'mae', 'num_leaves': 20, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.9}
    # Best CatBoost MAE score: 343.17189579386667
    cat_param =  {'verbose': False, 'subsample': 0.9, 'random_strength': 0.1, 'loss_function': 'MAE', 'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 5, 'colsample_bylevel': 0.7, 'bagging_temperature': 1}
    # Best XGBoost MAE score: 352.23548398423674
    xgb_param = {'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.2, 'eval_metric': 'mae', 'colsample_bytree': 0.9}


In [ ]:
class Pipeline:

    def __init__(self, model='XGB'):
        self.model_type = model
        if model == 'LGB':
            self.model = lgb.LGBMRegressor(
                #num_leaves = 16,
                #max_depth=5,
                #learning_rate = 0.01,
                #n_estimators=1000,
                #objective = "mae",
                **lgb_param
            )
        elif model == 'CatBoost':
            self.model = CatBoostRegressor(
                #iterations=1000,
                #learning_rate=0.01,
                #depth=4,
                **cat_param
            )
        else:
            self.model = xgb.XGBRegressor(
                #objective = 'reg:absoluteerror',
                #tree_method = 'gpu_hist',
                #colsample_bytree = 0.6,
                #gamma = 0.8,
                #learning_rate = 0.01,
                #max_depth = 5,
                #min_child_weight = 5,
                #n_estimators = 1000,
                #subsample = 0.7
                **xgb_param
            )

    def fit(self, X, y, X_val, y_val):
        self.model.fit(X, y.ravel(), eval_set=[(X_val, y_val.ravel())], verbose=False)

    def predict(self, X):
        return self.model.predict(X)

    def grid_search(self, X, y, X_eval, y_eval):
       pass

In [ ]:
def train(model_type):
    X = df_train.drop(['yield'], axis=1)
    y = df_train['yield']
    SKFs = KFold(n_splits=5, shuffle=True, random_state=1)
    losses = []
    pipelines = []
    for fold, (idx_tr, idx_vl) in enumerate(SKFs.split(X, y)):
        train_dataframe = df_train.iloc[idx_tr]
        dev_dataframe = df_train.iloc[idx_vl]

        # splits data to features and target
        X_train = train_dataframe.drop('yield', axis=1)
        y_train = train_dataframe['yield']
        X_dev = dev_dataframe.drop('yield', axis=1)
        y_dev = dev_dataframe['yield']

        # crates and fits a pipeline
        pipelineMy = Pipeline(model=model_type)
        pipelineMy.fit(X_train, y_train, X_dev, y_dev)

        # evaluates the model
        pipelines.append(pipelineMy)
        loss = mean_absolute_error(y_dev, pipelineMy.predict(X_dev))
        losses.append(loss)
        print(f'Fold {fold} loss: {loss}')
    print(f'Mean loss: {np.array(losses).mean()}')
    return losses, pipelines

In [ ]:
lossesLGB, pipelinesLGB = train('LGB')

In [ ]:
lossesCB, pipelinesCB = train('CatBoost')

In [ ]:
lossesXGB, pipelinesXGB = train('XGB')

# Make Submission

In [ ]:
preds = pipelinesLGB[0].predict(df_test)
for i in range(1, len(pipelinesLGB)):
    preds += pipelinesLGB[i].predict(df_test)
preds = preds / 5.0

In [ ]:
preds

In [ ]:
submission = pd.DataFrame()
submission['yield'] = preds
submission.index += 15289

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=True, header=True, index_label="id")